In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver



In [ ]:
load_dotenv(override=True)

In [3]:
class State(TypedDict):
    
    # A dictionary with the key 'messages' and with these annotations
    # This annotation tells LangGraph the type, and how to update it - the "reducer"
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [4]:
def push(text: str):
    """Send a push notification to the user"""
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

push_tool = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
)

In [5]:
import nest_asyncio
nest_asyncio.apply()
async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [6]:
navigate_tool = next(tool for tool in tools if tool.name == "navigate_browser")
extract_text_tool = next(tool for tool in tools if tool.name == "extract_text")
    
    # Navigate to CNN
await navigate_tool.arun({"url": "https://www.cnn.com"})
    
    # Extract text from the page
text = await extract_text_tool.arun({})

In [ ]:
text

In [6]:
all_tools = tools + [push_tool]

In [7]:
graph_builder = StateGraph(State)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


In [ ]:
memory = MemorySaver()

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=all_tools))
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))